In [111]:
using JuMP, GLPK
using DataFrames, CSV, XLSX

In [112]:
# PARAMETERS
F = 7       # number of fish types
G = 16       # number of fishing grounds
I = 53       # number of processing facilities
# J = 100     # number of cities
J = 50     # number of cities

50

In [113]:
# DATA: FISH 
df_fish = DataFrame(CSV.File("fish_data.csv"))
fish_price = df_fish."2021 Market Price"
fish_health_value = df_fish."Protein Value"

# DATA: FISHING GROUND
df_fishingground_max = DataFrame(CSV.File("fishingground_restriction.csv"))

# DATA: PROCESSING FACILITY
df_processingfacility = DataFrame(CSV.File("processing_facility.csv"))
processing_cap = df_processingfacility."Processing Capacity"
operational_cost = df_processingfacility."Operational Cost"

# DATA: CITIES
df_cities = DataFrame(CSV.File("cities_data.csv"))
cities_budget = df_cities."Average Yearly Budget of a Person to buy Fish"
cities_population = df_cities."Population (what year?)"
cities_min_consumption = df_cities."Minimum Fish Consumption"
cities_max_consumption = df_cities."Maximum Fish Consumption"

# DATA FOR OBJECTIVE FUNCTION
fishing_cost_3d = [
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 1")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 2")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 3")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 4")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 5")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 6")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 7")...)
]
transportation_cost_3d = [
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 1")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 2")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 3")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 4")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 5")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 6")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 7")...)
]

7-element Vector{DataFrame}:
 53×50 DataFrame
 Row │ 1        2        3        4        5        6        7        8        ⋯
     │ Any      Any      Any      Any      Any      Any      Any      Any      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 7.28397  12.0587  998.909  1129.66  2.51548  440.831  653.62   29.0764  ⋯
   2 │ 659.239  660.254  489.461  464.348  665.494  1018.26  18.2635  644.627
   3 │ 13.5408  16.782   981.008  1109.14  19.6201  454.003  633.065  17.4569
   4 │ 18.7134  15.7528  983.766  1108.08  22.6525  448.581  631.877  8.21751
   5 │ 901.306  902.844  384.425  221.357  907.705  1261.31  255.949  887.621  ⋯
   6 │ 15.4909  15.7021  1007.27  1137.5   8.75785  433.24   661.411  33.6992
   7 │ 11.9314  20.1038  980.106  1111.55  18.9111  457.577  635.623  25.3562
   8 │ 18.2363  28.1686  975.669  1110.62  24.7321  464.521  634.921  34.1243
   9 │ 2.40375  12.1728  993.54   1124.52  5.81916  445.625  648.513  26.8153  ⋯


In [114]:
# VARIABLES
m = Model(GLPK.Optimizer)
@variable(m, x[1:F, 1:G, 1:I])
@variable(m, y[1:F, 1:I, 1:J])
@variable(m, z[1:I], Bin)
@variable(m, s[1:J])

50-element Vector{VariableRef}:
 s[1]
 s[2]
 s[3]
 s[4]
 s[5]
 s[6]
 s[7]
 s[8]
 s[9]
 s[10]
 s[11]
 s[12]
 s[13]
 ⋮
 s[39]
 s[40]
 s[41]
 s[42]
 s[43]
 s[44]
 s[45]
 s[46]
 s[47]
 s[48]
 s[49]
 s[50]

In [115]:
# CONSTRAINTS
# Non-negativity constraints
for f in 1:F
    for g in 1:G
        for i in 1:I
            @constraint(m, x[f, g, i] >= 0)
        end
    end
end
for f in 1:F
    for i in 1:I
        for j in 1:J
            @constraint(m, y[f, i, j] >= 0)
        end
    end
end
for j in 1:J
    @constraint(m, s[j] >= 0)
end      

# Fishing Restriction Constraint
for f in 1:F
    for g in 1:G
        @constraint(m, sum(x[f, g, i] for i in 1:I) <= df_fishingground_max[g, f])
    end
end

# Processing Capacity Constraint
for i in 1:I
    @constraint(m, sum(y[f, i, j] for f in 1:F for j in 1:J) <= processing_cap[i])
end

# Protein Consumption Constraint 
for j in 1:J 
    @constraint(m, sum(y[f, i, j]*fish_health_value[f] for f in 1:F for i in 1:I) <= 43800 * cities_population[j])
    # @constraint(m, sum(y[f, i, j]*fish_health_value[f] for f in 1:F for i in 1:I) >= 17520 * cities_population[j])
    @constraint(m, sum(y[f, i, j]*fish_health_value[f] for f in 1:F for i in 1:I) >= 5840 * cities_population[j])
end

# Distribution Capacity Constraint
@constraint(m, sum(y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) <= sum(x[f, g, i] for f in 1:F for g in 1:G for i in 1:I))

# Facility Opening Constraint
# Part 1
for i in 1:I
    @constraint(m, sum(x[f,g,i] for f in 1:F for g in 1:G) <= sum(z[i] * df_fishingground_max[g, f] for f in 1:F for g in 1:G) )
end
# Part 2
for i in 1:I
    @constraint(m, sum(y[f,i,j] for f in 1:F for j in 1:J) <= z[i] * processing_cap[i])
end

# # Indicator variables
# # If we close facility i, we cannot send any fish to facility i
# for f in 1:F
#     for g in 1:G
#         for i in 1:I
#             @constraint(m, x[f,g,i] <= z[i] * df_fishingground_max[g, f])
#         end
#     end
# end

# # Indicator variables
# # If we close facility i, we cannot send any fish from facility i
# for f in 1:F
#     for i in 1:I
#         for j in 1:J
#             @constraint(m, y[f,i,j] <= z[i] * processing_cap[i])
#         end
#     end
# end



# Budget Constraint
for j in 1:J
    @constraint(m, sum(y[f, i, j] * fish_price[f] for f in 1:F for i in 1:I)  <= cities_population[j] * (0.15) * cities_budget[j] + s[j])
end

In [116]:
# OBJECTIVE FUNCTION
@objective(m, Max, (10^5)*sum(fish_health_value[f]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(fishing_cost_3d[f][g, i]*x[f, g, i] for f in 1:F for g in 1:G for i in 1:I) - sum(transportation_cost_3d[f][i,j]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(operational_cost[i]*z[i] for i in 1:I) - sum(s[j] for j in 1:J))

2.6352992716031518e7 y[1,1,1] + 2.635298794133492e7 y[1,1,2] + 2.6352001091238126e7 y[1,1,3] + 2.635187034112824e7 y[1,1,4] + 2.635299748451598e7 y[1,1,5] + 2.6352559169256907e7 y[1,1,6] + 2.635234637967522e7 y[1,1,7] + 2.6352970923629683e7 y[1,1,8] + 2.6352979667243194e7 y[1,1,9] + 2.635298089907676e7 y[1,1,10] + 2.6352086511566542e7 y[1,1,11] + 2.6352992212584812e7 y[1,1,12] + 2.635295957024842e7 y[1,1,13] + 2.6351788994410243e7 y[1,1,14] + 2.6352980709786076e7 y[1,1,15] + 2.635297643173857e7 y[1,1,16] + 2.6352995698004156e7 y[1,1,17] + 2.6352975578637075e7 y[1,1,18] + 2.6352985538709152e7 y[1,1,19] + 2.6352974643390246e7 y[1,1,20] + 2.6352434970924787e7 y[1,1,21] + 2.6352459409384094e7 y[1,1,22] + 2.635296379784304e7 y[1,1,23] + 2.635293911825518e7 y[1,1,24] + 2.6352332038431834e7 y[1,1,25] + 2.635297227472208e7 y[1,1,26] + 2.6352921738877982e7 y[1,1,27] + 2.635298145674361e7 y[1,1,28] + 2.63529648920751e7 y[1,1,29] + 2.635297698602376e7 y[1,1,30] + [[...24529 terms omitted...]] - s

In [117]:
optimize!(m)

In [118]:
solution_summary(m)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 1.13753e+17
  Objective bound    : 1.13753e+17
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.93190e+01


In [119]:
objective_value(m)

1.137527469735713e17

In [120]:
println(value.(x))

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 

In [121]:
# Print the optimal solution for each decision variable
println("Optimal solution for x:")
for f in 1:F
    for g in 1:G
        for i in 1:I
            println("x[$f, $g, $i] = ", value(x[f, g, i]))
        end
    end
end

println("\nOptimal solution for y:")
for f in 1:F
    for i in 1:I
        for j in 1:J
            println("y[$f, $i, $j] = ", value(y[f, i, j]))
        end
    end
end

println("\nOptimal solution for z:")
for i in 1:I
    println("z[$i] = ", value(z[i]))
end

println("\nOptimal solution for s:")
for j in 1:J
    println("s[$j] = ", value(s[j]))
end


Optimal solution for x:
x[1, 1, 1] = 0.0
x[1, 1, 2] = 0.0
x[1, 1, 3] = 0.0
x[1, 1, 4] = 0.0
x[1, 1, 5] = 0.0
x[1, 1, 6] = 0.0
x[1, 1, 7] = 0.0
x[1, 1, 8] = 0.0
x[1, 1, 9] = 0.0
x[1, 1, 10] = 0.0
x[1, 1, 11] = 0.0
x[1, 1, 12] = 0.0
x[1, 1, 13] = 0.0
x[1, 1, 14] = 0.0
x[1, 1, 15] = 0.0
x[1, 1, 16] = 0.0
x[1, 1, 17] = 0.0
x[1, 1, 18] = 0.0
x[1, 1, 19] = 0.0
x[1, 1, 20] = 0.0
x[1, 1, 21] = 0.0
x[1, 1, 22] = 0.0
x[1, 1, 23] = 0.0
x[1, 1, 24] = 0.0
x[1, 1, 25] = 0.0
x[1, 1, 26] = 0.0
x[1, 1, 27] = 0.0
x[1, 1, 28] = 0.0
x[1, 1, 29] = 0.0
x[1, 1, 30] = 0.0
x[1, 1, 31] = 0.0
x[1, 1, 32] = 0.0
x[1, 1, 33] = 0.0
x[1, 1, 34] = 0.0
x[1, 1, 35] = 0.0
x[1, 1, 36] = 0.0
x[1, 1, 37] = 0.0
x[1, 1, 38] = 0.0
x[1, 1, 39] = 0.0
x[1, 1, 40] = 0.0
x[1, 1, 41] = 0.0
x[1, 1, 42] = 0.0
x[1, 1, 43] = 0.0
x[1, 1, 44] = 0.0
x[1, 1, 45] = 0.0
x[1, 1, 46] = 0.0
x[1, 1, 47] = 0.0
x[1, 1, 48] = 0.0
x[1, 1, 49] = 0.0
x[1, 1, 50] = 0.0
x[1, 1, 51] = 0.0
x[1, 1, 52] = 0.0
x[1, 1, 53] = 0.0
x[1, 2, 1] = 0.0
x[1, 2, 2] = 0